In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [2]:
# Read stock symbols from CSV
csv_file = "stock_symbol.csv"
symbols_df = pd.read_csv(csv_file)

# Validate CSV data
if 'Symbol' not in symbols_df.columns:
    raise ValueError("CSV file must contain a 'Symbol' column.")

stock_symbols = symbols_df['Symbol'].tolist()


In [4]:

start_date = "2010-01-01"
end_date = pd.Timestamp.today()
data = yf.download(stock_symbols, start = start_date, end = end_date)
data.reset_index(inplace = True)
data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  6 of 6 completed


Price        Date       Close                                                  \
Ticker                   AAPL        AMZN        GOOG        MSFT        NFLX   
0      2010-01-04    6.440330    6.695000   15.536651   23.254057    7.640000   
1      2010-01-05    6.451466    6.734500   15.468232   23.261572    7.358571   
2      2010-01-06    6.348848    6.612500   15.078298   23.118807    7.617143   
3      2010-01-07    6.337109    6.500000   14.727282   22.878384    7.485714   
4      2010-01-08    6.379241    6.676000   14.923614   23.036158    7.614286   
...           ...         ...         ...         ...         ...         ...   
3830   2025-03-26  221.529999  201.130005  167.139999  389.970001  970.650024   
3831   2025-03-27  223.850006  201.360001  164.080002  390.579987  976.719971   
3832   2025-03-28  217.899994  192.720001  156.059998  378.799988  933.849976   
3833   2025-03-31  222.130005  190.259995  156.229996  375.390015  932.530029   
3834   2025-04-01  223.190002  192.169998  158.880005  382.190002  928.380005   

Price                     High                          ...        Open  \
Ticker        TSLA        AAPL        AMZN        GOOG  ...        GOOG   
0              NaN    6.455076    6.830500   15.605068  ...   15.541608   
1              NaN    6.487879    6.774000   15.563670  ...   15.547309   
2              NaN    6.477047    6.736500   15.514588  ...   15.514588   
3              NaN    6.379842    6.616000   15.121431  ...   15.106557   
4              NaN    6.379843    6.684000   14.954104  ...   14.675225   
...            ...         ...         ...         ...  ...         ...   
3830    272.059998  225.020004  206.009995  171.940002  ...  171.300003   
3831    273.130005  224.990005  203.789993  167.440002  ...  166.710007   
3832    263.549988  223.809998  199.259995  163.809998  ...  162.360001   
3833    259.160004  225.619995  191.330002  157.130005  ...  154.809998   
3834    268.459991  223.679993  193.929993  160.080002  ...  155.300003   

Price                                          Volume                        \
Ticker        MSFT        NFLX        TSLA       AAPL       AMZN       GOOG   
0        23.006114    7.931429         NaN  493729600  151998000   78541293   
1        23.178925    7.652857         NaN  601904800  177038000  120638494   
2        23.201454    7.361429         NaN  552160000  143576000  159744526   
3        23.013624    7.731429         NaN  477131200  220604000  257533695   
4        22.750648    7.498571         NaN  447610800  196610000  189680313   
...            ...         ...         ...        ...        ...        ...   
3830    395.000000  993.809998  282.660004   34466100   32855300   22498000   
3831    390.130005  974.219971  272.480011   37094800   27317700   21571200   
3832    388.079987  972.000000  275.579987   39818600   52548200   34866500   
3833    372.540009  920.530029  249.309998   65299300   63547600   33591600   
3834    374.649994  927.500000  263.799988   36377700   41208500   20094500   

Price                                    
Ticker      MSFT      NFLX         TSLA  
0       38409100  17239600          NaN  
1       49749600  23753100          NaN  
2       58182400  23290400          NaN  
3       50559700   9955400          NaN  
4       51197400   8180900          NaN  
...          ...       ...          ...  
3830    16108400   3527400  153629800.0  
3831    13766800   3047200  162572100.0  
3832    21632000   4422700  123809400.0  
3833    35184700   4634100  134008900.0  
3834    19661900   3516900  145924600.0  

[3835 rows x 31 columns]

In [5]:
# Dictionary to store results for each stock
predictions_dict = {}

for stock_symbol in stock_symbols:
    print(f"Processing stock: {stock_symbol}")
    data = yf.download(stock_symbol, start=start_date, end=end_date)
    data.reset_index(inplace=True)

    # Determine the cutoff point (80% for training, 20% for testing)
    cutoff = int(len(data) * 0.8)

    # Step 1: Prepare the training and testing data using iloc
    data_train = data.iloc[:cutoff][['Open', 'Close']].copy()
    data_test = data.iloc[cutoff:][['Open', 'Close']].copy()

    # Display the shapes of both datasets
    print(f"Total data points: {len(data)}")
    print(f"Training Data shape: {data_train.shape}")
    print(f"Testing Data shape: {data_test.shape}")


Processing stock: GOOG


[*********************100%***********************]  1 of 1 completed


Total data points: 3835
Training Data shape: (3068, 2)
Testing Data shape: (767, 2)
Processing stock: AAPL


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Total data points: 3835
Training Data shape: (3068, 2)
Testing Data shape: (767, 2)
Processing stock: MSFT
Total data points: 3835
Training Data shape: (3068, 2)
Testing Data shape: (767, 2)
Processing stock: AMZN


...

Total data points: 3835
Training Data shape: (3068, 2)
Testing Data shape: (767, 2)
Processing stock: TSLA


[*********************100%***********************]  1 of 1 completed


Total data points: 3713
Training Data shape: (2970, 2)
Testing Data shape: (743, 2)
Processing stock: NFLX


[*********************100%***********************]  1 of 1 completed

Total data points: 3835
Training Data shape: (3068, 2)
Testing Data shape: (767, 2)


In [6]:
# Step 2: Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_train_scaled = scaler.fit_transform(data_train)

In [7]:
# Step 3: Prepare the input (x) and output (y) for the LSTM model
x = []
y = []

# Create sequences of data
time_steps = 60  # Number of time steps to look back

for i in range(time_steps, data_train_scaled.shape[0]):
    x.append(data_train_scaled[i - time_steps:i])  # Previous 60 time steps
    y.append(data_train_scaled[i])  # Current Open and Close prices

x, y = np.array(x), np.array(y)

In [8]:
# Step 5: Build the LSTM model
model = Sequential()

model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

# Output layer to predict both Open and Close prices
model.add(Dense(units=2))  # 2 units for Open and Close prices

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Step 6: Train the model
model.fit(x, y, epochs=100, batch_size=32)  # Adjust epochs and batch size as needed

Epoch 1/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - loss: 0.0495
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - loss: 0.0058
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - loss: 0.0051
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - loss: 0.0044
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - loss: 0.0044
Epoch 6/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - loss: 0.0039
Epoch 7/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - loss: 0.0043
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - loss: 0.0032
Epoch 9/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - loss: 0.0037
Epoch 10/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - loss: 0.0030
Epoch 11/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/step - loss: 0.0030
Epoch 12/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - loss: 0.0029
Epoch 13/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - loss: 0.0031
Epoch 14/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - loss: 0.0029
Epoch 15/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - l

In [10]:
# Step 7: Prepare the test data (you can do this in a similar way)
data_test_scaled = scaler.transform(data_test)

x_test = []
for i in range(time_steps, data_test_scaled.shape[0]):
    x_test.append(data_test_scaled[i - time_steps:i])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

In [11]:
# Step 8: Make predictions
predictions = model.predict(x_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step


In [12]:
# Step 9: Inverse transform the predictions to get actual prices
predictions_inverse = scaler.inverse_transform(predictions)

In [13]:
# predictions_inverse will contain two columns: [Predicted Open, Predicted Close]
predicted_open_price = predictions_inverse[:, 0]
predicted_close_price = predictions_inverse[:, 1]

In [14]:
# Save the predicted prices in the dictionary
predictions_dict[stock_symbol] = {
    "Predicted_Open_Prices": predicted_open_price,
    "Predicted_Close_Prices": predicted_close_price
}

In [15]:
# Save the trained model for each stock
model.save(f'stock_price_model_GOOG.h5')
model.save(f'stock_price_model_AAPL.h5')
model.save(f'stock_price_model_MSFT.h5')
model.save(f'stock_price_model_AMZN.h5')
model.save(f'stock_price_model_TSLA.h5')
model.save(f'stock_price_model_NFLX.h5')

In [16]:
# Save predictions to a CSV file
for symbol, prediction in predictions_dict.items():
    df = pd.DataFrame({
        'Predicted_Open': prediction['Predicted_Open_Prices'],
        'Predicted_Close': prediction['Predicted_Close_Prices']
    })
    df.to_csv(f'predictions_GOOG.csv', index=False)
    df.to_csv(f'predictions_AAPL.csv', index=False)
    df.to_csv(f'predictions_MSFT.csv', index=False)
    df.to_csv(f'predictions_AMZN.csv', index=False)
    df.to_csv(f'predictions_TSLA.csv', index=False)
    df.to_csv(f'predictions_NFLX.csv', index=False)

print("All models trained and predictions saved.")

All models trained and predictions saved.
